(optional)

In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')

In [ ]:
# os.chdir('...')

# **HW5: Brain signal classification**
In *HW 5*, you need to finish:

1.  Model Implementation Part: Implement LSTM and EEGNet models to predict the label of each samples.

2.  Model Competition Part: Implementing a model to reach better accuracy performance.

In [ ]:
import numpy as np
import os
import math
import csv
import matplotlib.pyplot as plt
# Import the packages you need here
import tensorflow as tf
from tensorflow.keras import layers, models, constraints
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
data = np.load('data.npz')
label = np.load('label.npz')

In [ ]:
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']

Y_train = label['Y_train']
Y_val = label['Y_val']

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
def change_dim(X):
    temp = []
    for i in range(len(X)):
        temp.append(X[i].T.copy())
    return np.array(temp)

X_train = change_dim(X_train)
X_val = change_dim(X_val)
X_test = change_dim(X_test)

# (number of samples, time steps, number of features)
print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
Y_train.shape, Y_val.shape

In [ ]:
Y_train = tf.keras.utils.to_categorical(Y_train, 6)
Y_val = tf.keras.utils.to_categorical(Y_val, 6)

print(Y_train.shape, Y_val.shape)

## Model Implementation Part

### LSTM

In [ ]:
# Build your model here:
lstm = models.Sequential()

lstm.add(layers.LSTM(units=64, input_shape=(200, 22), dropout=0.2))

lstm.add(layers.Dense(units=6, activation='softmax'))

lstm.summary()

In [ ]:
lstm.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

lstm_train_history = lstm.fit(X_train, Y_train,
                          validation_data=(X_val, Y_val),
                          batch_size=64,
                          epochs=100,
                          verbose=1)

In [ ]:
pred_train = lstm.evaluate(X_train, Y_train, verbose=0)
pred_val = lstm.evaluate(X_val, Y_val, verbose=0)

print("Train loss:", pred_train[0])
print("Train accuracy:", pred_train[1])
print("Validation loss:", pred_val[0])
print("Validation accuracy:", pred_val[1])

plt.plot(lstm_train_history.history['accuracy'])
plt.plot(lstm_train_history.history['val_accuracy'])
plt.title('Train History')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

plt.plot(lstm_train_history.history['loss'])
plt.plot(lstm_train_history.history['val_loss'])
plt.title('Train History')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
pred_test = lstm.predict(X_test)
pred_test = np.argmax(pred_test, axis = 1)
pred_test = pred_test.reshape((-1, 1))

In [ ]:
output = pred_test
assert(output.shape == (190, 1))
np.savetxt('lstm_output.csv', output, delimiter=",", fmt='%i')

### EEGNet

In [ ]:
X_train = change_dim(X_train).reshape((X_train.shape[0], 22, 200, 1))
X_val = change_dim(X_val).reshape((X_val.shape[0], 22, 200, 1))
X_test = change_dim(X_test).reshape((X_test.shape[0], 22, 200, 1))

# (number of samples, number of features(channel size), time steps)
print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
F1 = 16
F2 = 16
C = 22
D = 2
p = 0.25

# Build your model here:
eegnet = models.Sequential([
    # block1
    layers.Conv2D(filters=F1, kernel_size=(1, 64), padding='same', input_shape=(22, 200, 1), use_bias=False),
    layers.BatchNormalization(),
    layers.DepthwiseConv2D(kernel_size=(C, 1), padding='valid', depth_multiplier=D, depthwise_constraint=constraints.max_norm(1.), use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('elu'),
    layers.AveragePooling2D(pool_size=(1, 4)),
    layers.Dropout(p),
    # # block2
    layers.SeparableConv2D(filters=F2, kernel_size=(1, 16), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('elu'),
    layers.AveragePooling2D(pool_size=(1, 8)),
    layers.Dropout(p),
    layers.Flatten(),
    layers.Dense(units=6, kernel_constraint=constraints.max_norm(0.25)),
    layers.Activation('softmax')
])

eegnet.summary()

In [ ]:
eegnet.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

eegnet_train_history = eegnet.fit(X_train, Y_train,
                          validation_data=(X_val, Y_val),
                          batch_size=64,
                          epochs=100,
                          verbose=1)

In [ ]:
pred_train = eegnet.evaluate(X_train, Y_train, verbose=0)
pred_val = eegnet.evaluate(X_val, Y_val, verbose=0)

print("Train loss:", pred_train[0])
print("Train accuracy:", pred_train[1])
print("Validation loss:", pred_val[0])
print("Validation accuracy:", pred_val[1])

plt.plot(eegnet_train_history.history['accuracy'])
plt.plot(eegnet_train_history.history['val_accuracy'])
plt.title('Train History')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

plt.plot(eegnet_train_history.history['loss'])
plt.plot(eegnet_train_history.history['val_loss'])
plt.title('Train History')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
pred_test = eegnet.predict(X_test)
pred_test = np.argmax(pred_test, axis = 1)
pred_test = pred_test.reshape((-1, 1))

In [ ]:
output = pred_test
assert(output.shape == (190, 1))
np.savetxt('eegnet_output.csv', output, delimiter=",", fmt='%i')

## Model Competition Part

In [ ]:
# Build your model here:


In [ ]:
output = "..."
assert(output.shape == (190, 1))
np.savetxt('competition_output.csv', output, delimiter=",")